# Taylor series expansion of the rovibrational kinetic energy operator for a triatomic molecule using the Eckart frame

In this example, we compute Taylor series expansion coefficients for the rovibrational kinetic energy $G$-matrix and pseudopotential of a triatomic molecule (water) in terms of valence coordinates, in the molecular frame defined by the Eckart conditions.

In [1]:
import itertools

import jax
import jax.numpy as jnp
import numpy as np
from scipy.special import factorial

from vibrojet.eckart import eckart, EckartMethod
from vibrojet.keo import Gmat, batch_Gmat, batch_pseudo, pseudo
from vibrojet.taylor import deriv_list

jax.config.update("jax_enable_x64", True)

Use Taylor-mode automatic differentiation to compute expansion of $G$-matrix.

In [2]:
# Masses of O, H, H atoms
masses = np.array([15.9994, 1.00782505, 1.00782505])

# Equilibrium values of valence coordinates
r1, r2, alpha = 0.958, 0.958, 1.824
q0 = jnp.array([r1, r2, alpha], dtype=jnp.float64)


# Valence-to-Cartesian coordinate transformation
#   input: array of three valence coordinates
#   output: array of shape (number of atoms, 3) containing Cartesian coordinates of atoms
# `eckart` rotates coordinates to the Eckart frame
@eckart(q0, masses, method=EckartMethod.quaternion)
def valence_to_cartesian(q):
    r1, r2, a = q
    return jnp.array(
        [
            [0.0, 0.0, 0.0],
            [r1 * jnp.sin(a / 2), 0.0, r1 * jnp.cos(a / 2)],
            [-r2 * jnp.sin(a / 2), 0.0, r2 * jnp.cos(a / 2)],
        ]
    )


# Generate list of multi-indices specifying the integer exponents for each
# coordinate in the Taylor series expansion
max_order = 4  # max total expansion order
deriv_ind = [
    elem
    for elem in itertools.product(*[range(0, max_order + 1) for _ in range(len(q0))])
    if sum(elem) <= max_order
]
print("max expansion order:", max_order)
print("number of expansion terms:", len(deriv_ind))

# Function for computing kinetic G-matrix for given masses of atoms
# and internal coordinates
func = lambda x: Gmat(x, masses, valence_to_cartesian)

# Compute Taylor series expansion coefficients
Gmat_coefs = deriv_list(func, deriv_ind, q0, if_taylor=True)

print(Gmat_coefs.shape)  # (number of expansion terms, ncoo+3+3, ncoo+3+3)

max expansion order: 4
number of expansion terms: 35
Time for d= 0 : 3.14 s
Time for d= 1 : 0.02 s
Time for d= 2 : 5.82 s
Time for d= 3 : 10.01 s
Time for d= 4 : 15.83 s
(35, 9, 9)


Optionally, compare the results with computationally more intensive calculations using nested `jacfwd` calls.

In [3]:
def jacfwd(x0, ind):
    f = func
    for _ in range(sum(ind)):
        f = jax.jacfwd(f)
    i = sum([(i,) * o for i, o in enumerate(ind)], start=tuple())
    return f(x0)[:, :, *i]


Gmat_coefs2 = np.array([jacfwd(q0, ind) / np.prod(factorial(ind)) for ind in deriv_ind])

print("max difference for G-matrix:", np.max(np.abs(Gmat_coefs - Gmat_coefs2)))

max difference for G-matrix: 5.737632591262809e-12


Compute expansion of pseudopotential.

In [4]:
max_order = 2  # max total expansion order
deriv_ind = [
    elem
    for elem in itertools.product(*[range(0, max_order + 1) for _ in range(len(q0))])
    if sum(elem) <= max_order
]
print("max expansion order:", max_order)
print("number of expansion terms:", len(deriv_ind))

func = lambda x: pseudo(x, masses, valence_to_cartesian)

pseudo_coefs = deriv_list(func, deriv_ind, q0, if_taylor=True)

max expansion order: 2
number of expansion terms: 10
Time for d= 0 : 49.73 s
Time for d= 1 : 0.02 s
Time for d= 2 : 105.22 s


Optionally, compare the results with calculations using nested `jacfwd` calls.

In [5]:
def jacfwd(x0, ind):
    f = func
    for _ in range(sum(ind)):
        f = jax.jacfwd(f)
    i = sum([(i,) * o for i, o in enumerate(ind)], start=tuple())
    return f(x0)[*i]


pseudo_coefs2 = np.array(
    [jacfwd(q0, ind) / np.prod(factorial(ind)) for ind in deriv_ind]
)

print(
    "max difference for pseudopotential:", np.max(np.abs(pseudo_coefs - pseudo_coefs2))
)

max difference for pseudopotential: 1.3713474800169934e-12


Example of computing $G$-matrix and pseudopotential on a batch of coordinate values.

In [6]:
# Generate grid of coordinates
r1_arr = np.linspace(r1 - 0.5, r1 + 1, 10)
r2_arr = np.linspace(r2 - 0.5, r2 + 1, 10)
alpha_arr = np.linspace(alpha - 40 * np.pi / 180, alpha + 40 * np.pi / 180, 10)
xa, xb, xc = np.meshgrid(r1_arr, r2_arr, alpha_arr, indexing="ij")
x = np.column_stack([xa.ravel(), xb.ravel(), xc.ravel()])

Gmat_vals = batch_Gmat(x, masses, valence_to_cartesian)
print(Gmat_vals.shape)  # (number of points, ncoo+3+3, ncoo+3+3)

pseudo_vals = batch_pseudo(x, masses, valence_to_cartesian)
print(pseudo_vals.shape)  # (number of points, )

(1000, 9, 9)
(1000,)
